In [1]:
cd ..

C:\Users\pi96doc\Documents\Programming\PythonScripts\StateModeling


In [2]:
# %matplotlib notebook
# %matplotlib inline
#import mpld3
#mpld3.enable_notebook()

import StateModeling as stm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from Corona.LoadData import loadData, preprocessData
from Corona.CoronaModel import CoronaModel, plotTotalCases
from bokeh.io import push_notebook, show, output_notebook

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
# import cufflinks as cf

output_notebook()

xpos = np.arange(0,100)/100.0

AllMeasured = loadData(r"COVID-19 Linelist 2020_04_27.xlsx", useThuringia = True, pullData=False)
# AllMeasured = preprocessData(AllMeasured)
# AllMeasured = loadData(useThuringia = False, pullData=False)
ExampleRegions = ['SK Jena', 'LK Greiz'] # 'SK Gera',
AllMeasured = preprocessData(AllMeasured, ReduceDistricts=ExampleRegions, SumDistricts=False, SumAges=True, SumGender=True)

M = CoronaModel(AllMeasured)

if False:
    mobdat = AllMeasured['mobility']
    mobdate = mobdat['date'].to_numpy()
    plt.figure('Retail and recreation');plt.plot(mobdat['retail_and_recreation_percent_change_from_baseline'].to_numpy())
    offsetDay=0; plt.xticks(range(offsetDay, len(mobdate), 7), [date for date in mobdate[offsetDay:-1:7]], rotation="vertical")
    plt.ylabel('Percent Change'); plt.tight_layout()

Tmax = 120

M.toFit(['r0', 'h', 'aT0', 'aBase', 'I0', 'd', 'rd', 'T0', 'q']) # 'q',
if AllMeasured['Cases'].shape[-2] > 1:
    M.toFit.append(['Age Border', 'Age Sigma'])

PopSum = np.sum(AllMeasured['Population'])
measured = AllMeasured['Cases'][:,  np.newaxis, :, :, :] / PopSum
measuredDead = AllMeasured['Dead'][:, np.newaxis, :, :, :] / PopSum
xlim = None  # (60,100)
# fittedVars, fittedRes = M.fit({'detected': measured}, Tmax, otype=otype, oparam=oparam, NIter=NIter, verbose=True, lossScale=lossScale)
FitDict = {'cases': measured}
if "Hospitalized" in AllMeasured.keys():
    FitDict['hospitalization'] = AllMeasured['Hospitalized'][:, np.newaxis, :, :, :]/ PopSum
FitDict['deaths'] = measuredDead

# SimDict = {'cases': None, 'cumul_cases': None, 'cumul_dead':None}

if True:
    otype = "L-BFGS"
    lossScale = 1.0  # 1e4
    oparam = {"normFac": 'max'}
else:
    lossScale = None
    otype = "nesterov"  # "adagrad"  "adadelta" "SGD" "nesterov"  "adam"
    learnrate = {"nesterov": 1000.0, "adam": 7e-7}
    oparam = {"learning_rate": tf.constant(learnrate[otype], dtype=stm.CalcFloatStr)}
# oparam['noiseModel'] = 'Poisson'
oparam['noiseModel'] = 'Gaussian'
# oparam['noiseModel'] = 'ScaledGaussian'  # is buggy? Why the NaNs?

# tf.config.experimental_run_functions_eagerly(True)

def doFit(NIter=0):
    fittedVars, fittedRes = M.fit(FitDict, Tmax, otype=otype, oparam=oparam, NIter=NIter, verbose=True, lossScale=lossScale)

doFit()
# simulated = M.simulate('simulated', FitDict, Tmax=Tmax)

#def showSimRes(ymin=0.0001,ymax=1.0):
#    ylim = [ymin,ymax]
#    M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='occupancy', 
#                  xlim=xlim, dims=("District"), Dates=AllMeasured['Dates'], 
#                  legendPlacement='upper right', styles=['.','-','--'], ylim=ylim, figsize=[10,5])
#    plt.tight_layout()
#    plt.show()
def showSimRes(ymin=0.0001,ymax=1.0):
    doFit()
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='cases',
                  legendPlacement='upper right',figsize=[10,5], Dates=AllMeasured['Dates'])
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='hospitalization',
                  legendPlacement='upper right',figsize=[10,5], Dates=AllMeasured['Dates'])
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='deaths', 
                  legendPlacement='upper right',figsize=[10,5], Dates=AllMeasured['Dates'])
    return p
    

def showSimStates(ymin=1.0,ymax=1.0):
    ylim = [-ymin,ymax]
    M.showStates(MinusOne=('S'), dims2d=None, Dates = AllMeasured['Dates'], ylim=ylim, figsize=[10,5])
    plt.tight_layout()
    plt.show()
    #M.assignNewVar('r0',r0)
    #M.assignNewVar('h',h)
    #simulated = M.simulate('simulated', FitDict, Tmax=Tmax)

def assignParam(d=0.06):
    aDict = {'d':d}
    for varN, newval in aDict.items():
        M.assignNewVar(varN, newval)
        print(varN+": "+str(newval))
    doFit()
    print('Loss: '+str(M.FitResultVars['Loss'].numpy()))



Loading BokehJS ...

tracing doBuildModel
tracing traceModel
tracing time step 119
 .. done

Loss is: 5.8213605e-05


In [3]:
M.DataDict={}
g = M.getGUI(showResults=showSimRes, doFit=doFit)

HBox()

Loss is: 5.8213605e-05
replotting: measured_cases_SK Jena
replotting: measured_cases_LK Greiz
replotting: Fit_cases_SK Jena
replotting: Fit_cases_LK Greiz
showing figure


replotting: measured_hospitalization_SK Jena
replotting: measured_hospitalization_LK Greiz
replotting: Fit_hospitalization_SK Jena
replotting: Fit_hospitalization_LK Greiz
showing figure


replotting: measured_deaths_SK Jena
replotting: measured_deaths_LK Greiz
replotting: Fit_deaths_SK Jena
replotting: Fit_deaths_LK Greiz
showing figure


looking for figures: 
Thuringia_deaths
looking for figures: 
Figure(id='1002', ...)
cleared renderer of figure _figure_cases named: _figure_cases
looking for figures: 
was plotted
looking for figures: 
ColumnDataSource(id='1036', ...)
looking for figures: 
was plotted
looking for figures: 
ColumnDataSource(id='1070', ...)
looking for figures: 
was plotted
looking for figures: 
ColumnDataSource(id='1105', ...)
looking for figures: 
was plotted
looking for figures: 
ColumnDataSource(id='1135', ...)
looking for figures: 
looking for figures: 
Figure(id='1357', ...)
cleared renderer of figure _figure_hospitalization named: _figure_hospitalization
looking for figures: 
was plotted
looking for figures: 
ColumnDataSource(id='1391', ...)
looking for figures: 
was plotted
looking for figures: 
ColumnDataSource(id='1425', ...)
looking for figures: 
was plotted
looking for figures: 
ColumnDataSource(id='1460', ...)
looking for figures: 
was plotted
looking for figures: 
ColumnDataSource(id='1490'

Loss is: 3.971188e-05
Cumul: set useBars to False
Updating y-data of: measured_cases_SK Jena
Cumul: set useBars to False
Updating y-data of: measured_cases_LK Greiz
Cumul: set useBars to False
Updating y-data of: Fit_cases_SK Jena
Cumul: set useBars to False
Updating y-data of: Fit_cases_LK Greiz
pushing notebook
Cumul: set useBars to False
Updating y-data of: measured_hospitalization_SK Jena
Cumul: set useBars to False
Updating y-data of: measured_hospitalization_LK Greiz
Cumul: set useBars to False
Updating y-data of: Fit_hospitalization_SK Jena
Cumul: set useBars to False
Updating y-data of: Fit_hospitalization_LK Greiz
pushing notebook
Cumul: set useBars to False
Updating y-data of: measured_deaths_SK Jena
Cumul: set useBars to False
Updating y-data of: measured_deaths_LK Greiz
Cumul: set useBars to False
Updating y-data of: Fit_deaths_SK Jena
Cumul: set useBars to False
Updating y-data of: Fit_deaths_LK Greiz
pushing notebook
looking for figures: 
Thuringia_deaths
looking for figu

In [4]:
M.DataDict

{'_title': 'Thuringia_deaths',
 '_figure_cases': Figure(id='1002', ...),
 '_measured_cases_SK Jena': 'was plotted',
 'measured_cases_SK Jena': ColumnDataSource(id='1036', ...),
 '_measured_cases_LK Greiz': 'was plotted',
 'measured_cases_LK Greiz': ColumnDataSource(id='1070', ...),
 '_Fit_cases_SK Jena': 'was plotted',
 'Fit_cases_SK Jena': ColumnDataSource(id='1105', ...),
 '_Fit_cases_LK Greiz': 'was plotted',
 'Fit_cases_LK Greiz': ColumnDataSource(id='1135', ...),
 '_figure_cases_notebook_handle': <bokeh.io.notebook.CommsHandle at 0x228c049dcc0>,
 '_figure_hospitalization': Figure(id='1357', ...),
 '_measured_hospitalization_SK Jena': 'was plotted',
 'measured_hospitalization_SK Jena': ColumnDataSource(id='1391', ...),
 '_measured_hospitalization_LK Greiz': 'was plotted',
 'measured_hospitalization_LK Greiz': ColumnDataSource(id='1425', ...),
 '_Fit_hospitalization_SK Jena': 'was plotted',
 'Fit_hospitalization_SK Jena': ColumnDataSource(id='1460', ...),
 '_Fit_hospitalization_LK G

In [9]:
M.DataDict['_figure_cases'].renderers[0].name

'measured_cases_SK Jena'

In [28]:
#interact_manual(showSim,
#        ymin=ymin,
#        ymax=ymax)

d = widgets.FloatLogSlider(0.06,min=-10,max=2.0,continuous_update=False)
r = widgets.FloatLogSlider(0.01,min=-10,max=2.0,continuous_update=False)
uiS = widgets.HBox((d,r))
allSimWidgets = {'d':d}
print('Simulation Control:')
    
d.observe(assignParam, names='value')
# d.observe(showSimRes, names='value')
# outS = widgets.interactive_output(assignParam, allSimWidgets)
display(uiS, outS)

ymin = widgets.FloatLogSlider(0.001,min=-10,max=3,continuous_update=False)
ymax = widgets.FloatLogSlider(30.0,min=-10,max=3,continuous_update=False)
ui = widgets.HBox((ymin,ymax))
allWidgets = {'ymin': ymin, 'ymax': ymax}
out = widgets.interactive_output(showSimRes, allWidgets)

# out.layout.width = '700px';out.layout.height = '350px'
display(ui, out)

ymin = widgets.FloatLogSlider(0.0001,min=-10,max=6.0,continuous_update=False)
ymax = widgets.FloatLogSlider(0.01,min=-10,max=2.0,continuous_update=False)
ui = widgets.HBox((ymin,ymax))
allWidgets = {'ymin': ymin, 'ymax': ymax}
out2 = widgets.interactive_output(showSimStates, allWidgets)
#out2.layout.width = '700px';out2.layout.height = '350px'
#display(ui, out2)


Simulation Control:


Output(outputs=({'output_type': 'stream', 'text': 'd: 0.039810717055349734\nLoss is: tf.Tensor(0.000104670384,…

Output()

AttributeError: dtype

In [5]:
from datetime import total_seconds
total_seconds()*1000

ImportError: cannot import name 'total_seconds' from 'datetime' (C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\envs\StateModeling\lib\datetime.py)

In [8]:
from ipywidgets import widgets, Layout
from IPython.display import display
item_layout = Layout(
    display='flex',flex_flow='row',
    justify_content='space-between'
)
box_layout=Layout(
    display='flex',flex_flow='column',
    border='solid 2px',align_items='stretch',width='50%')
tickLayout = Layout(display='flex', width='30%')
inFitWidget = widgets.Checkbox(value=True, indent=False, layout=tickLayout, description='Country')
drop = widgets.Dropdown(options=['a','b'], indent=False, value='a')
dropWidget = widgets.HBox((inFitWidget, drop), display='flex', layout = item_layout)
valueWidget = widgets.FloatLogSlider(value=1.0,base=10,min=-7,max=1)
boxWidget = widgets.HBox((Label('Hi'),valueWidget),layout=item_layout)
# valueWidget = widgets.HBox((inFitWidget,valueWidget))
widget = widgets.Box((dropWidget, boxWidget), layout=box_layout)
display(widgets.HBox((widget,widget,widget, widget)))

In [15]:
valueWidget.description

''

In [10]:
def showSimRes(ymin=0.0001,ymax=1.0):
    doFit()
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='cases',
                  legendPlacement='upper right',figsize=[10,5], Dates=AllMeasured['Dates'])
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='hospitalization',
                  legendPlacement='upper right',figsize=[10,5], Dates=AllMeasured['Dates'])
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='deaths', 
                  legendPlacement='upper right',figsize=[10,5], Dates=AllMeasured['Dates'])
    return p


In [5]:
g['T0'].children[1].value

0

In [18]:
M.Var['T0']()

<tf.Tensor: shape=(1, 2, 1, 1), dtype=float32, numpy=
array([[[[2.5118864e-06]],

        [[7.1999998e+00]]]], dtype=float32)>

In [4]:
from ipywidgets import interact
import numpy as np
import pandas as pd

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

x = np.linspace(0, 2*np.pi, 20)
y = np.sin(x)
Dates = M.getDates(AllMeasured['Dates'],y)
Dates = pd.to_datetime(Dates, dayfirst=True)

source = ColumnDataSource(data=dict(x=x, y=y))

p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
           background_fill_color='#efefef', name='Blubb') # , x_axis_type='datetime'
# r = p.line(pd.to_datetime(Dates), y, color="#8888cc", line_width=1.5, alpha=0.8)
# r = p.vbar_stack([x], y=[y-0.5], color="#8888cc")
r = p.vbar('x', top='y',color="#cc8800", alpha=0.4, source=source, name='Hi there')
# r = p.vbar(x, top=y+1, width=0.15, color="#8888cc", alpha=0.6)
p.xaxis.axis_label = 'Hi'
q=show(p, notebook_handle=True)
# pd.date_range(start=Dates[0], periods=toPlot.shape[0]).map(lambda x: x.strftime('%d.%m.%Y'))


In [48]:
# pd.to_datetime(Dates)
Dates = pd.date_range(start='14.02.2020', periods=x.shape[0]).map(lambda x: x.strftime('%d.%m.%Y'))

In [6]:
type(p)

bokeh.plotting.figure.Figure

In [14]:
from bokeh.io.notebook import CommsHandle
isinstance(q,CommsHandle)

True

In [44]:
type(p)

bokeh.plotting.figure.Figure